# Database Engineering

In [1]:
#import dependencies
import pandas as pd
from datetime import date

#create an engine that will interact with the SQL database
from sqlalchemy import create_engine
#create a session factory to bound to the engine the base and class defintions
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Date, Integer, String, Float

#PyMySQL
import pymysql
pymysql.install_as_MySQLdb()



In [2]:
#Use the engine and connection string to create a database called hawaii.sqlite
engine=create_engine("sqlite:///hawaii.sqlite",echo=True)
conn = engine.connect()

2018-04-11 22:11:42,927 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-11 22:11:42,931 INFO sqlalchemy.engine.base.Engine ()
2018-04-11 22:11:42,934 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-11 22:11:42,935 INFO sqlalchemy.engine.base.Engine ()


In [3]:
#create the measurement class
    
class Measurment(Base):
    __tablename__="measurments"
    
    id=Column(Integer, primary_key=True)
    station=Column(String(255))
    date=Column(Date)
    prcp=Column(Float)
    tobs=Column(Integer)

#Class for Station
class Station(Base):
    __tablename__="stations"
    
    id=Column(Integer, primary_key=True)
    station=Column(String(255))
    name=Column(String(255))
    latitude=Column(Float)
    longitude=Column(Float)
    elevation=Column(Float)

In [4]:
#create datanase add the classes


# create tables
Base.metadata.create_all(conn)

session=Session(bind=engine)

2018-04-11 22:11:57,966 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurments")
2018-04-11 22:11:57,968 INFO sqlalchemy.engine.base.Engine ()
2018-04-11 22:11:57,971 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stations")
2018-04-11 22:11:57,972 INFO sqlalchemy.engine.base.Engine ()
2018-04-11 22:11:57,973 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE measurments (
	id INTEGER NOT NULL, 
	station VARCHAR(255), 
	date DATE, 
	prcp FLOAT, 
	tobs INTEGER, 
	PRIMARY KEY (id)
)


2018-04-11 22:11:57,974 INFO sqlalchemy.engine.base.Engine ()
2018-04-11 22:11:58,196 INFO sqlalchemy.engine.base.Engine COMMIT
2018-04-11 22:11:58,198 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE stations (
	id INTEGER NOT NULL, 
	station VARCHAR(255), 
	name VARCHAR(255), 
	latitude FLOAT, 
	longitude FLOAT, 
	elevation FLOAT, 
	PRIMARY KEY (id)
)


2018-04-11 22:11:58,199 INFO sqlalchemy.engine.base.Engine ()
2018-04-11 22:11:58,328 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
#Use Pandas to read  cleaned measurements and stations CSV data.
csv_path="Resources/clean_hawaii_measurments.csv"
csv_path2="Resources/clean_hawaii_stations.csv"

clean_measurments=pd.read_csv(csv_path)
clean_stations=pd.read_csv(csv_path2)

In [6]:
clean_measurments.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [7]:
clean_stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [8]:
#send the station dataframe to sqllite database
clean_stations.to_sql("stations", engine,if_exists='append',index=False)


#check to make sure the data successfully entered
df=pd.read_sql_query('Select * From stations',engine)
df.head()


2018-04-11 22:12:25,328 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stations")
2018-04-11 22:12:25,329 INFO sqlalchemy.engine.base.Engine ()
2018-04-11 22:12:25,332 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-11 22:12:25,334 INFO sqlalchemy.engine.base.Engine INSERT INTO stations (station, name, latitude, longitude, elevation) VALUES (?, ?, ?, ?, ?)
2018-04-11 22:12:25,335 INFO sqlalchemy.engine.base.Engine (('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0), ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6), ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0), ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9), ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6), ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5), ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9), ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI

,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [9]:
#send the station dataframe to sqlite database
clean_measurments.to_sql("measurments",engine,if_exists='append',index=False)

#check to make sure data successfully entered
df=pd.read_sql_query('Select * From measurments limit 3',engine)
df.head()

2018-04-11 22:13:12,977 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurments")
2018-04-11 22:13:12,978 INFO sqlalchemy.engine.base.Engine ()
2018-04-11 22:13:12,982 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-04-11 22:13:13,059 INFO sqlalchemy.engine.base.Engine INSERT INTO measurments (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-04-11 22:13:13,060 INFO sqlalchemy.engine.base.Engine (('USC00519397', '2010-01-01', 0.08, 65), ('USC00519397', '2010-01-02', 0.0, 63), ('USC00519397', '2010-01-03', 0.0, 74), ('USC00519397', '2010-01-04', 0.0, 76), ('USC00519397', '2010-01-07', 0.06, 70), ('USC00519397', '2010-01-08', 0.0, 64), ('USC00519397', '2010-01-09', 0.0, 68), ('USC00519397', '2010-01-10', 0.0, 73)  ... displaying 10 of 18103 total bound parameter sets ...  ('USC00516128', '2017-08-22', 0.5, 76), ('USC00516128', '2017-08-23', 0.45, 76))
2018-04-11 22:13:13,111 INFO sqlalchemy.engine.base.Engine COMMIT
2018-04-11 22:13:13,381 INFO sqlalchemy.engine.base

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65
1,2,USC00519397,2010-01-02,0.00,63
2,3,USC00519397,2010-01-03,0.00,74
